In [15]:
# Choose a native GUI backend that doesn't need Jupyter widgets
import sys, platform, importlib, warnings
import matplotlib

def _force_backend(candidates):
    for name in candidates:
        try:
            if name == "Qt5Agg":
                importlib.import_module("PyQt5")  # ensure Qt is installed
            if name == "QtAgg":  # Matplotlib 3.8+ alias
                importlib.import_module("PyQt5")
            if name == "TkAgg":
                import tkinter  # ensure Tk is present
            matplotlib.use(name, force=True)
            return name
        except Exception as e:
            warnings.warn(f"Backend {name} failed: {e}")
    raise RuntimeError("No suitable GUI backend found. Install Tk (TkAgg) or PyQt5 (Qt5Agg/QtAgg).")

# Try modern Qt first, then Tk, then older Qt spellings
chosen = _force_backend(["QtAgg", "Qt5Agg", "TkAgg", "MacOSX"] if sys.platform == "darwin"
                        else ["QtAgg", "Qt5Agg", "TkAgg"])
print("Matplotlib backend =>", matplotlib.get_backend())


C:\Users\Asylum User\AppData\Local\Temp\ipykernel_932\2811665825.py:17: UserWarning: Backend QtAgg failed: 
    Importing PyQt5 disabled by IPython, which has
    already imported an Incompatible QT Binding: pyside6
    
  warnings.warn(f"Backend {name} failed: {e}")
C:\Users\Asylum User\AppData\Local\Temp\ipykernel_932\2811665825.py:17: UserWarning: Backend Qt5Agg failed: 
    Importing PyQt5 disabled by IPython, which has
    already imported an Incompatible QT Binding: pyside6
    
  warnings.warn(f"Backend {name} failed: {e}")
C:\Users\Asylum User\AppData\Local\Temp\ipykernel_932\2811665825.py:17: UserWarning: Backend TkAgg failed: Cannot load backend 'TkAgg' which requires the 'tk' interactive framework, as 'qt' is currently running
  warnings.warn(f"Backend {name} failed: {e}")


RuntimeError: No suitable GUI backend found. Install Tk (TkAgg) or PyQt5 (Qt5Agg/QtAgg).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.widgets import Slider

def interactive_vector_angle_native(ux, uy, uz, U_mag=None, window=5, origin='upper', cmap='viridis'):
    ux, uy, uz = map(np.asarray, (ux, uy, uz))
    if U_mag is None or getattr(U_mag, "shape", None) != ux.shape:
        U_mag = np.sqrt(ux**2 + uy**2 + uz**2)

    assert ux.shape == uy.shape == uz.shape == U_mag.shape, "All arrays must have the same shape"
    assert window % 2 == 1, "window must be odd"

    nrows, ncols = ux.shape
    half = window // 2

    # --- component scales (controlled by sliders) ---
    scale = {'ux': 1.0, 'uy': 1.0, 'uz': 1.0}

    def clamp_rc(r, c):
        return int(np.clip(r, 0, nrows-1)), int(np.clip(c, 0, ncols-1))

    def scaled_fields():
        """Return scaled components and their magnitude."""
        sux = scale['ux'] * ux
        suy = scale['uy'] * uy
        suz = scale['uz'] * uz
        smag = np.sqrt(sux**2 + suy**2 + suz**2)
        return sux, suy, suz, smag

    def avg_vec(rc):
        sux, suy, suz, _ = scaled_fields()
        r, c = clamp_rc(*rc)
        r0, r1 = max(0, r-half), min(nrows, r+half+1)
        c0, c1 = max(0, c-half), min(ncols, c+half+1)
        return np.array([sux[r0:r1, c0:c1].mean(),
                         suy[r0:r1, c0:c1].mean(),
                         suz[r0:r1, c0:c1].mean()], float)

    def angle_deg(v1, v2):
        n1, n2 = np.linalg.norm(v1), np.linalg.norm(v2)
        if n1 == 0 or n2 == 0:
            return float("nan")
        return float(np.degrees(np.arccos(np.clip(np.dot(v1, v2) / (n1 * n2), -1.0, 1.0))))

    def angle_map_from_v1(v1):
        sux, suy, suz, smag = scaled_fields()
        n1 = np.linalg.norm(v1)
        if n1 == 0:
            return np.full_like(U_mag, np.nan, dtype=float)
        dot = v1[0]*sux + v1[1]*suy + v1[2]*suz
        with np.errstate(invalid='ignore', divide='ignore'):
            cosang = np.clip(dot / (n1 * smag), -1.0, 1.0)
            ang = np.degrees(np.arccos(cosang))
        ang[~np.isfinite(ang)] = np.nan
        return ang

    # defaults
    coord1 = [nrows//2, max(0, ncols//4)]
    coord2 = [nrows//2, min(ncols-1, (3*ncols)//4)]

    fig, ax = plt.subplots()
    plt.subplots_adjust(bottom=0.23)  # room for sliders

    # --- initialize backdrop using scaled fields with default scales ---
    v1_init = avg_vec(coord1)
    ang_img = angle_map_from_v1(v1_init)

    im = ax.imshow(ang_img, origin=origin, cmap=cmap, vmin=0, vmax=180)
    im.cmap.set_bad(alpha=0.0)
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label("Angle from p1 (°)")

    p1, = ax.plot(coord1[1], coord1[0], marker='x', markersize=9, mew=2, linestyle='None')
    p2, = ax.plot(coord2[1], coord2[0], marker='x', markersize=9, mew=2, linestyle='None')
    c1 = Circle((coord1[1], coord1[0]), radius=window/2, fill=False, lw=2)
    c2 = Circle((coord2[1], coord2[0]), radius=window/2, fill=False, lw=2)
    ax.add_patch(c1); ax.add_patch(c2)

    selected = None
    dragging = False
    result = {'coord1': tuple(coord1), 'coord2': tuple(coord2),
              'v1': None, 'v2': None, 'angle_deg': None}

    def title_text(ang, v1):
        s = scale
        return (f"Angle diff (p1 vs p2) = {ang:.2f}°   |   "
                f"p1={tuple(coord1)}  p2={tuple(coord2)}   |   "
                f"scales: ux={s['ux']:.2f}, uy={s['uy']:.2f}, uz={s['uz']:.2f}")

    def update():
        v1, v2 = avg_vec(coord1), avg_vec(coord2)
        ang = angle_deg(v1, v2)
        result.update(coord1=tuple(coord1), coord2=tuple(coord2), v1=v1, v2=v2, angle_deg=ang)
        im.set_data(angle_map_from_v1(v1))
        ax.set_title(title_text(ang, v1))
        fig.canvas.draw_idle()

    def move_artists():
        p1.set_data([coord1[1]], [coord1[0]])
        p2.set_data([coord2[1]], [coord2[0]])
        c1.center = (coord1[1], coord1[0])
        c2.center = (coord2[1], coord2[0])
        update()

    pick_thresh2 = 9  # ~3 px radius
    move_artists()

    def on_press(event):
        nonlocal selected, dragging
        if event.inaxes != ax or event.button != 1 or event.xdata is None or event.ydata is None:
            return
        r, c = clamp_rc(int(round(event.ydata)), int(round(event.xdata)))
        d1 = (r - coord1[0])**2 + (c - coord1[1])**2
        d2 = (r - coord2[0])**2 + (c - coord2[1])**2
        if d1 <= d2 and d1 <= pick_thresh2:
            selected = 'p1'
        elif d2 < d1 and d2 <= pick_thresh2:
            selected = 'p2'
        else:
            if d1 > d2:
                coord1[:] = [r, c]; selected = 'p1'
            else:
                coord2[:] = [r, c]; selected = 'p2'
            move_artists()
        dragging = True

    def on_motion(event):
        if not dragging or selected is None or event.inaxes != ax or event.xdata is None or event.ydata is None:
            return
        r, c = clamp_rc(int(round(event.ydata)), int(round(event.xdata)))
        if selected == 'p1': coord1[:] = [r, c]
        else:                coord2[:] = [r, c]
        move_artists()

    def on_release(event):
        nonlocal selected, dragging
        if event.button == 1:
            dragging = False
            selected = None

    def on_key(event):
        if event.key == 'r':
            coord1[:] = [nrows//2, max(0, ncols//4)]
            coord2[:] = [nrows//2, min(ncols-1, (3*ncols)//4)]
            move_artists()

    cid1 = fig.canvas.mpl_connect('button_press_event', on_press)
    cid2 = fig.canvas.mpl_connect('motion_notify_event', on_motion)
    cid3 = fig.canvas.mpl_connect('button_release_event', on_release)
    cid4 = fig.canvas.mpl_connect('key_press_event', on_key)

    # --- sliders for scaling each component ---
    ax_sux = fig.add_axes([0.12, 0.14, 0.76, 0.03])
    ax_suy = fig.add_axes([0.12, 0.09, 0.76, 0.03])
    ax_suz = fig.add_axes([0.12, 0.04, 0.76, 0.03])

    s_sux = Slider(ax_sux, 'scale ux', -2.0, 2.0, valinit=1.0)
    s_suy = Slider(ax_suy, 'scale uy', -2.0, 2.0, valinit=1.0)
    s_suz = Slider(ax_suz, 'scale uz', -2.0, 2.0, valinit=1.0)

    def on_scale(_):
        scale['ux'] = s_sux.val
        scale['uy'] = s_suy.val
        scale['uz'] = s_suz.val
        update()

    s_sux.on_changed(on_scale)
    s_suy.on_changed(on_scale)
    s_suz.on_changed(on_scale)

    ax.set_xlabel("Column (x)")
    ax.set_ylabel("Row (y)")
    plt.show()

    try:
        for cid in (cid1, cid2, cid3, cid4):
            fig.canvas.mpl_disconnect(cid)
    except Exception:
        pass

    return result


In [16]:
import h5py
import os

from matplotlib.colors import hsv_to_rgb
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.pyplot as plt
import numpy as np

file_loc = r'D:\User Data\Ralph\3DPFM_Test\PolyPZT'
base_filename = 'Image'
log_filename = 'Log_3DPFM'
os.chdir(file_loc)

with (h5py.File(base_filename+'.hf5', 'r')) as f:
    ux = f['3DPFM/ux'][:]
    uy = f['3DPFM/uy'][:]
    uz = f['3DPFM/uz'][:]
    #topo = f['C/Channel_000/Channel_000/Channel_000'][:]
    #scale = f['C/Channel_000/Channel_000/original_metadata'].attrs['FastScanSize']/f['C/Channel_000/Channel_000/original_metadata'].attrs['PointsLines']

U_mag = np.sqrt(np.abs(ux)**2+np.abs(uy)**2+np.abs(uz)**2)
U_mag_max = min(U_mag.max(), 3e-9)

In [17]:
# Use your arrays (np.real() if needed)
res = interactive_vector_angle_native(np.real(ux), np.real(uy), np.real(uz), U_mag=None, window=5, cmap = 'jet')
#res = interactive_vector_angle_native(np.sign(np.real(ux))*np.abs(ux),np.sign(np.real(uy))*np.abs(uy), np.sign(np.real(uz))*np.abs(uz), U_mag=None, window=5, cmap = 'jet')
print("coord1:", res['coord1'])
print("coord2:", res['coord2'])
print("v1:", res['v1'])
print("v2:", res['v2'])
print(f"angle: {res['angle_deg']:.2f}°")


coord1: (21, 249)
coord2: (42, 236)
v1: [-1.30839549e-11 -8.54729135e-12 -4.67918933e-12]
v2: [-3.06587568e-12  8.13096899e-12  1.52155753e-11]
angle: 110.60°
